## The Unit Commitment Problem

The unit commitment problem is one of the most import linear optimisation problems of them all, without it we would not have the electricity we need to power our homes, our industries, our way of life. 

The idea of this problem is relatively simple. We have an expected electricity $d_{i}$ demand for the next $n$ periods and that demand must be satisfied. To satisfy the demand in each period, we must have power generation stations (of which there are $m$) running to generate the electricity required. At all times we wish the energy generated to match (balance) the energy required. Each unit of energy costs a different amount, depending on the station that is running. Each station can generate electricity up until some limit. However, switching stations on is not a trivial affair. It incurs some fixed cost to ramp a station up, and it will not come online immediately.

The objective function in this case will be to generate the energy required while minimising costs. Many objectives can be important here, such as minimising the number of stations ramped up, maximising the stability of the system under uncertain demands, among others. Consider the plot below, where we assume that we know with perfect clarity the demand for each one-hour period:

![title](images/demand.png)

We can see that the energy demand peaks in the morning (when everyone gets ready to go to work) and in the evening (when everyone gets home from work). This is an example of a bi-modal distribution. 

We will assume that the demand does not fluctuate in between the hours (or maybe we can assume that it is enough to model the average demand over an entire hour!). This means that we must make sure, at the beginning of each hour that there is enough electricity being generated at each station to satsify the demand.

| Station | Is Operating at Start? ($q_{i,0}$) | Cost to Ramp-up ($f_{i}$) | Per-Hour Cost to Operate ($l_{i}$) | Cost to Generate One Unit ($g_{i}$) | Electricty Generation Limit ($k_{i}$)|
| --- | --- | --- | --- | --- | --- |
| $S_{1}$ | No | 43000 | 2300 | 3658 | 4 kW |
| $S_{2}$ | Yes | 13000 | 3400 | 4421 | 7 kW |
| $S_{3}$ | No | 21000 | 6700 | 1658 | 4 kW |
| $S_{4}$ | No | 103000 | 1200 | 358 | 9 kW |

Consider a situation where we have four generation stations, $S_{1}, S_{2}, S_{3}$ and $S_{4}$. At the start of the first time period, only station $S_{2}$ is ramped-up (the others are switched off). At the start of the first time period, the demand is being met, or $S_{2}$ is generating $4$ kW of electricity. The cost for station $S_{i}$ to generate one unit ($1$ kW) of electricity is $g_{i}$. The fixed cost to ramp-up any station is $f_{i}$. Each station can generate electricity in the range $[0, k_{i}]$. If we choose to ramp-up a station $S_{i}$, it can (for simplicity) generate electricity immediately. We will not consider that we can ramp them down. Running any station (even while producing no electricity) incurs a standing cost of $l_{t}$ every time period. 

Given the data above, how can we formulate a mixed integer programme that will allow us to minimise the total operating costs?

## First, a Simplified Problem

The point of this exercise is not to understand unit commitment problems, but to understand how to model situations for which there is some degree of conditional occurence. Consider only the demand at 19:00, which is $19$ kW. Let us assume that, previous to this hour, only S2 is active. We only need to satisfy the electricity demand for this hour. 

Which stations should we ramp-up to minimise the cost of meeting the demand?

Well, first we need a continuous variable $x$ to model the electricity generated at this time slot. The demand, $d_{t}$ is equal to $19 $kW. For each station $S_{i}$, we have a continuous variable $y_{i}$ that models the electricity generation at that station. We also have a binary variable $r_{i}$, which allows us to decide whether or not we have ramped-up a station to generate energy at this time slot. A station is active if a binary variable $q_{i}$ takes non-zero value.

Now, if we want to minimise the total cost of producing the electricity, we must take into account the generation costs, the ramping-up costs and the operating costs for each station, that is, our objective consists of the components. First the generation costs:

$$ \sum_{i=1}^{m}g_{i}y_{i} = g_{1}y_{1} + g_{2}y_{2} + g_{3}y_{3} + g_{4}y_{4},$$

then the ramping-up costs:

$$ \sum_{i=1}^{m}f_{i}r_{i} = f_{1}r_{1} + f_{2}r_{2} + f_{3}r_{3} + f_{4}r_{4},$$

and finally the opertaing costs:

$$ \sum_{i=1}^{m}l_{i}q_{i} = l_{1}q_{1} + l_{2}q_{2} + l_{3}q_{3} + l_{4}q_{4}.$$

This gives a full objective function:

$$ Z = \sum_{i=1}^{m}g_{i}y_{i} + \sum_{i=1}^{m}f_{i}r_{i} + \sum_{i=1}^{m}l_{i}q_{i}.$$

Next, we must make sure that the electricy generated meets demand, so we devise the foloowing constraint:

$$x = d_{t}.$$

We must also make sure that generation at all the stations sums to the global generation:

$$x = y_{1} + y_{2} + y_{3} + y_{4}.$$

We then ensure that generation can only occur at a station if it is active:

$$y_{i} \leq q_{i}k_{i} \quad \quad \forall i \in \{1,...,4\}.$$

We also make sure that a station is only active if it has been ramped up. Since we know station two is ramped up, we need to have a way of modelling that. We will say that a station is active if we have just ramped it up, or if it was active in the previous time step. For that reason, we have additional modelling variables, $\hat{q}_{i}$, for each station, that inform us if a station was previously active. Then we can model the activity variables with the following constraints:

$$ q_{i} \geq \frac{1}{2}(r_{i} + \hat{q}_{i}).$$

In other words, the activity is forced to take a value greater than zero if the station was previously active or if it was ramped up, but since each $q_{i}$ is binary, this means that it must take the value one.

We also force the condition that all $\hat{q}_{i}$ are zero-valued, except for $\hat{q}_{2}$, since we know that station was active. This gives us the full model:

$$ Z^{*} = \quad \text{minimise} \quad Z = \sum_{i=1}^{m}g_{i}y_{i} + \sum_{i=1}^{m}f_{i}r_{i} + \sum_{i=1}^{m}l_{i}q_{i} \quad \text{subject to:}$$

$$x = d_{t},$$
$$x = y_{1} + y_{2} + y_{3} + y_{4},$$
$$y_{i} \leq q_{i}k_{i} \quad \quad \forall i \in \{1,...,4\},$$
$$ q_{i} \geq \frac{1}{2}(r_{i} + \hat{q}_{i}),$$
$$ \hat{q}_{i} = 0 \quad \forall i \in \{1, 3, 4 \},$$
$$ \hat{q}_{2} = 1 \quad \forall i \in \{1, 3, 4 \},$$
$$ q_{i} \in  \{0,1\} \quad \forall i \in \{1,...,4\},$$
$$ r_{i} \in  \{0,1\} \quad \forall i \in \{1,...,4\},$$
$$ y_{i} \in  [0, k_{i}] \quad \forall i \in \{1,...,4\},$$
$$ x \in [0, d_{t}].$$

## The Mixed Integer Linear Programme

Now, if we want to model the full problem...

First we need some continuous variables $x_{t}$ that model the electricity that is generated at the start of each period $t$. Next, we need a continuous variable $y_{it}$ that models the electricity generated at each station $i$ at the start of each time period $t$. Next, we need a binary variable r_{it} for each station for each time period that determines whether or not we have ramped up the station $S_{i}$ at time $t$. 

This allows us to to model all the decisions that we have made, but we can construct more variables for modelling pruposes. For example, all though we have a decision variable that tracks when we switch on a certain station, we will need another one to keep track of whether or not a station is currently active, so that even multiple time slots after it has been switched on, we can know. Therefore, we introduce the binary variable $q_{it}$ that determines whether or not a station $S_{i}$ is active at time period $t$.

The costs accumlated are the total costs proportional to the generation at each station: 

$$\sum_{t=1}^{n}\sum_{i=1}^{m}g_{i}y_{it},$$

plus the costs of ramping up the stations:

$$ \sum_{t=1}^{n}\sum_{i=1}^{m}r_{it}f_{i},$$

plus the costs of operating the stations:

$$ \sum_{t=1}^{n}\sum_{i=1}^{m}q_{it}l_{i}.$$

Now we must model all of the constraints. First, we make sure that the total generation at any given time period is equal to the energy generated in all stations:

$$ x_{it} = \sum_{i}^{m}y_{it} \quad \quad \forall t \in \{1, ..., n\}.$$

We must also ensure that energy can only be generated at a station if it is active:

$$ y_{it} \leq k_{i}q_{it} \quad \quad \forall i \in \{1, ..., m\}, t \in \{1,..., n\}.$$

We make sure that a station can only be ramped up once:

$$ \sum_{t=1}^{n}r_{it} \leq 1 \quad \quad \forall i \in \{1,...,m\}$$

We ensure that once a station has been ramped up, it stays active:

$$ q_{it} \geq \frac{1}{2}(q_{i,t-1} + r_{i,t-1}) \quad \quad i \in \{1, ..., m\}, t \in \{1,..., n\}.$$

We must also take care of the boundary conditions. We need to ensure that, at the beginning, there are variables to keep of the initial conditions. For this we require that $q_{2,1} = 1 = r_{2,1}$ and that $y_{2,1} = 4$. For this, we define two additional variables whose values we fix:

$$q_{2,0} = 1,$$
$$r_{2,0} = 1.$$

This makes sure that we don't need to make any adjustments to the last set of constraints to set the boundary conditions.

### The Mixed Integer Linear Programme (MILP)

$$Z^{*} = \text{minimise} \quad Z = \sum_{t=1}^{n}\sum_{i=1}^{m}g_{i}y_{it} + \sum_{t=1}^{n}\sum_{i=1}^{m}r_{it}f_{i} + \sum_{t=1}^{n}\sum_{i=1}^{m}q_{it}l_{i} \quad \text{subject to:}$$

$$ x_{it} = \sum_{i}^{m}y_{it} \quad \quad \forall t \in \{1, ..., n\},$$

$$ y_{it} \leq k_{i}q_{it} \quad \quad \forall i \in \{1, ..., m\}, t \in \{1,..., n\},$$


$$ \sum_{t=1}^{n}r_{it} \leq 1 \quad \quad \forall i \in \{1,...,m\},$$


$$ q_{it} \geq \frac{1}{2}(q_{i,t-1} + r_{i,t-1}) \quad \quad i \in \{1, ..., m\}, t \in \{1,..., n\},$$

$$ x_{t} \in [0, d_{t}] \quad \quad \forall t \in \{1, ..., n\},$$

$$ y_{it} \in [0, k_{i}] \quad \quad \forall i \in \{1, ..., m\}, t \in \{1, ..., n\},$$

$$ r_{it} \in \{0, 1\} \quad \quad \forall i \in \{1, ..., m\}, t \in \{1, ..., n\},$$

$$ q_{it} \in \{0, 1\} \quad \quad \forall i \in \{1, ..., m\}, t \in \{1, ..., n\}.$$

### Solving the Problem with Python

In [4]:
import xpress as xp

import matplotlib.pyplot as plt

Using the Community license in this session. If you have a full Xpress license, first set the XPRESS environment variable to the directory containing the license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPRESS environment variable to

/home/james/Environments/Kharkov/lib/python3.8/site-packages/xpress/license



In [53]:
generation_caps = {
    1: 4,
    2: 7,
    3: 4,
    4: 9,
}

rampup_costs = {
    1: 43000,
    2: 13000,
    3: 21000,
    4: 10300,   
}

generation_costs = {
    1: 3658,
    2: 4421,
    3: 1658,
    4: 358,
}

operating_costs = {
    1: 2300,
    2: 3400,
    3: 6700,
    4: 1200,
}

demand = {
    1: 4,
    2: 4,
    3: 5,
    4: 5,
    5: 5,
    6: 6,
    7: 9,
    8: 12,
    9: 15,
    10: 14,
    11: 12,
    12: 8,
    13: 8,
    14: 9,
    15: 10,
    16: 9,
    17: 8,
    18: 8,
    19: 12,
    20: 19,
    21: 16,
    22: 13,
    23: 9,
    24: 5,
}

hour = [
    "00:00",
    "01:00",
    "02:00",
    "03:00",
    "04:00",
    "05:00",
    "06:00",
    "07:00",
    "08:00",
    "09:00",
    "10:00",
    "11:00",
    "12:00",
    "13:00",
    "14:00",
    "15:00",
    "16:00",
    "17:00",
    "18:00",
    "19:00",
    "20:00",
    "21:00",
    "22:00",
    "23:00",
]

stations = range(1,5)

times = range(1,25)

In [143]:
problem  = xp.problem("Unit Commitment Problem") # Initialise the problem

x = {} # Initialise the empty dictionary for the global generation variables
y = {} # Initialise the empty dictionary for the local generation variables
r = {} # Initialise the empty dictionary for the station ramping variables
q = {} # Initialise the empty dictionary for the station activity variables

for time in times:
    dictionary_key = f"x_{time}" # Construct the dictionary key-string
    variable = xp.var(
        name=dictionary_key, # Name the variable
        lb=0, # At least zero electricity is produced (We could actually set this to be the same as the upper bound)
        ub=demand[time], # Make sure that the generation can't exceed the demand
        vartype=xp.continuous # Make the variable continuous
    )
    x[dictionary_key] = variable # Put the key in the dictionary

for station in stations: # Second the local generation variables
    for time in times:
        dictionary_key = f"y_{station},{time}" # Construct the dictionary key-string
        variable = xp.var(
            name=dictionary_key, # Name the variable
            lb=0, # At least zero electricity is produced (We could actually set this to be the same as the upper bound)
            ub=generation_caps[station], # Make sure that the generation can't exceed the station capacity
            vartype=xp.continuous # Make the variable continuous
        )
        y[dictionary_key] = variable # Put the key in the dictionary
        
for station in stations: # Third the station ramping variables
    for time in times:
        dictionary_key = f"r_{station},{time}" # Construct the dictionary key-string
        variable = xp.var(
            name=dictionary_key, # Name the variable
            vartype=xp.binary # Make the variable continuous
        )
        r[dictionary_key] = variable # Put the key in the dictionary
        
for station in stations: # Fourth the station activity variables
    for time in times:
        dictionary_key = f"q_{station},{time}" # Construct the dictionary key-string
        variable = xp.var(
            name=dictionary_key, # Name the variable
            vartype=xp.binary # Make the variable continuous
        )
        q[dictionary_key] = variable # Put the key in the dictionary
        
# Next we build variables to handle the boundary conditions

r["r_1,0"] = xp.var(name="r_1,0", lb=0, ub=0, vartype=xp.binary) # Station one is not ramped up at time zero
r["r_2,0"] = xp.var(name="r_2,0", lb=0, ub=0, vartype=xp.binary) # Station two is not ramped up at time zero
r["r_3,0"] = xp.var(name="r_3,0", lb=0, ub=0, vartype=xp.binary) # Station three is not ramped up at time zero
r["r_4,0"] = xp.var(name="r_4,0", lb=0, ub=0, vartype=xp.binary) # Station four is not ramped up at time zero
q["q_1,0"] = xp.var(name="q_1,0", lb=0, ub=0, vartype=xp.binary) # Station one is not active up at time zero
q["q_2,0"] = xp.var(name="q_2,0", lb=1, ub=1, vartype=xp.binary) # Station two is already active at time zero
q["q_3,0"] = xp.var(name="q_3,0", lb=0, ub=0, vartype=xp.binary) # Station three is not active at time zero
q["q_4,0"] = xp.var(name="q_4,0", lb=0, ub=0, vartype=xp.binary) # Station four is not active at time zero

# Next we add the variables to the problem

problem.addVariable(x) # Add the global generation variables
problem.addVariable(y) # Add the local generation variables
problem.addVariable(r) # Add the station ramping variables
problem.addVariable(q) # Add the station activity variables

# Now we build the objective function, we will do it in three parts, the generation, ramping and operating costs

generation_cost = sum([
    y[f"y_{station},{time}"] * generation_costs[station] for station in stations for time in times
]) # Multiply each station generating cost by the amount generated at each station at each time
ramping_cost = sum([
    y[f"y_{station},{time}"] * rampup_costs[station] for station in stations for time in times
]) # Multiply each station ramping cost by whether or not the station is ramped up at each station at each time
operation_cost = sum([
    y[f"y_{station},{time}"] * operating_costs[station] for station in stations for time in times
]) # Multiply each station operating cost by whether or not the station is active at each station at each time
objective_function = generation_cost + ramping_cost + operation_cost

problem.setObjective(objective_function, sense=xp.minimize)

# Now we build the the constraints

for time in times: # Make sure all the global productions meet the total demand 
    demand_satisfaction_constraint = x[f"x_{time}"] == demand[time]
    problem.addConstraint(demand_satisfaction_constraint)

for time in times: # Make sure all the local productions sum to the total production 
    local_generation_sum = sum([
        y[f"y_{station},{time}"] for station in stations
    ])
    demand_satisfaction_constraint = local_generation_sum == x[f"x_{time}"]
    problem.addConstraint(demand_satisfaction_constraint)
    
for time in times: # Make sure generation can only occur if the station is active 
    for station in stations:
        activity_constraint = y[f"y_{station},{time}"] <= generation_caps[station] * q[f"q_{station},{time}"]
        problem.addConstraint(activity_constraint)
        
for station in stations: # Make sure that a station can be ramped up at most once
    ramping_sum = sum([
        r[f"r_{station},{time}"] for time in times
    ])
    ramping_constraint = ramping_sum <= 1
    problem.addConstraint(ramping_constraint)   
    
for time in times: # Make sure that once a station is ramped up, it is marked as active with the q variables
    for station in stations:
        previous_rampings = sum([
            r[f"r_{station},{other_time}"] for other_time in range(0, time)
        ]) + q[f"q_{station},{0}"]
        activity_variable = q[f"q_{station},{time}"]
        activity_constraint = activity_variable == previous_rampings
        problem.addConstraint(activity_constraint)
        print(activity_variable, previous_rampings, activity_constraint)
        
initial_constraint = y["y_2,1"] == 4 # Set the initial generation to four kilo-Watts
problem.addConstraint(initial_constraint)
#initial_constraint = q["q_1,0"] == 0 # Station is not ramped up at previous time step
#problem.addConstraint(initial_constraint)
#initial_constraint = q["q_2,0"] == 0 # Station is not ramped up at previous time step
#problem.addConstraint(initial_constraint)
#initial_constraint = q["q_3,0"] == 0 # Station is not ramped up at previous time step
#problem.addConstraint(initial_constraint)
#initial_constraint = q["q_4,0"] == 0 # Station is not ramped up at previous time step
#problem.addConstraint(initial_constraint)
#initial_constraint = q["q_1,0"] == 0 # Station is not ramped up at previous time step
#problem.addConstraint(initial_constraint)
initial_constraint = q["q_2,0"] == 1 # Station is not ramped up at previous time step
problem.addConstraint(initial_constraint)
#initial_constraint = q["q_3,0"] == 0 # Station is not ramped up at previous time step
#problem.addConstraint(initial_constraint)
#initial_constraint = q["q_2,0"] == 0 # Station is not ramped up at previous time step
#problem.addConstraint(initial_constraint)

problem.solve() # Solve the Problem


q_1,1   r_1,0 + q_1,0 R3988
q_2,1   r_2,0 + q_2,0 R3989
q_3,1   r_3,0 + q_3,0 R3990
q_4,1   r_4,0 + q_4,0 R3991
q_1,2   r_1,1 + r_1,0 + q_1,0 R3992
q_2,2   r_2,1 + r_2,0 + q_2,0 R3993
q_3,2   r_3,1 + r_3,0 + q_3,0 R3994
q_4,2   r_4,1 + r_4,0 + q_4,0 R3995
q_1,3   r_1,1 + r_1,2 + r_1,0 + q_1,0 R3996
q_2,3   r_2,1 + r_2,2 + r_2,0 + q_2,0 R3997
q_3,3   r_3,1 + r_3,2 + r_3,0 + q_3,0 R3998
q_4,3   r_4,1 + r_4,2 + r_4,0 + q_4,0 R3999
q_1,4   r_1,1 + r_1,2 + r_1,3 + r_1,0 + q_1,0 R4000
q_2,4   r_2,1 + r_2,2 + r_2,3 + r_2,0 + q_2,0 R4001
q_3,4   r_3,1 + r_3,2 + r_3,3 + r_3,0 + q_3,0 R4002
q_4,4   r_4,1 + r_4,2 + r_4,3 + r_4,0 + q_4,0 R4003
q_1,5   r_1,1 + r_1,2 + r_1,3 + r_1,4 + r_1,0 + q_1,0 R4004
q_2,5   r_2,1 + r_2,2 + r_2,3 + r_2,4 + r_2,0 + q_2,0 R4005
q_3,5   r_3,1 + r_3,2 + r_3,3 + r_3,4 + r_3,0 + q_3,0 R4006
q_4,5   r_4,1 + r_4,2 + r_4,3 + r_4,4 + r_4,0 + q_4,0 R4007
q_1,6   r_1,1 + r_1,2 + r_1,3 + r_1,4 + r_1,5 + r_1,0 + q_1,0 R4008
q_2,6   r_2,1 + r_2,2 + r_2,3 + r_2,4 + r_2,5 + r_2